# LeNet 学习笔记

## 环境
我使用的设备环境：
```
                    c.'          bdim404@BdimMacBook-Pro.local 
                 ,xNMM.          ----------------------------- 
               .OMMMMo           OS: macOS 14.5 23F79 arm64 
               lMM"              Host: MacBookPro18,4 
     .;loddo:.  .olloddol;.      Kernel: 23.5.0 
   cKMMMMMMMMMMNWMMMMMMMMMM0:    Uptime: 14 days, 31 mins 
 .KMMMMMMMMMMMMMMMMMMMMMMMWd.    Packages: 293 (nix-user) 
 XMMMMMMMMMMMMMMMMMMMMMMMX.      Shell: bash 5.2.26 
;MMMMMMMMMMMMMMMMMMMMMMMM:       Resolution: 3024x1964 
:MMMMMMMMMMMMMMMMMMMMMMMM:       DE: Aqua 
.MMMMMMMMMMMMMMMMMMMMMMMMX.      WM: Quartz Compositor 
 kMMMMMMMMMMMMMMMMMMMMMMMMWd.    WM Theme: Blue (Dark) 
 'XMMMMMMMMMMMMMMMMMMMMMMMMMMk   Terminal: Apple_Terminal 
  'XMMMMMMMMMMMMMMMMMMMMMMMMK.   Terminal Font: Monaco 
    kMMMMMMMMMMMMMMMMMMMMMMd     CPU: Apple M1 Max 
     ;KMMMMMMMWXXWMMMMMMMk.      GPU: Apple M1 Max 
       "cooc*"    "*coo'"        Memory: 29725MiB / 65536MiB 

```
我使用的 python 版本是 3.11.0 并且是通过 nix 安装的。

参考 pytorch 官网的链接进行安装 [Installing on macOS](https://pytorch.org/get-started/locally/)
pip3 install torch torchvision torchaudio

In [1]:
!pip3 install torch torchvision torchaudio


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 定义批量大小为64，这是在训练过程中每次传递给神经网络的样本数量
batch_size = 64

# 数字的种类数量为10
num_classes = 10

# 定义学习率为0.001 
learning_rate = 0.001

# 定义训练轮数为10，这意味着整个数据集将被传递给神经网络10次
num_epochs = 10

# 定义了将在 Apple 设备使用 GPU 进行训练
device = torch.device('mps')

In [3]:
# 加载 MNIST 数据集
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True, # '表示我们需要加载训练数据集
                                           transform = transforms.Compose([ # transform 的作用是定义了对图像进行预处理的方式
                                                  transforms.Resize((32,32)), # 将图像大小调整为32x32像素。原始MNIST图像是28x28像素，这里我们将其增加到与CIFAR-10数据集相同的尺寸以方便后续处理
                                                  transforms.ToTensor(), # 将图像转换为PyTorch Tensor格式，并将像素值缩放到[0, 1]之间
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]), # 标准化图像数据。这里使用了MNIST训练和测试集的均值和标准差，以确保数据在不同的批次中有相似的分布。
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)

# 创建了一个PyTorch DataLoader对象，它提供了从训练数据集中按批加载和迭代数据的功能。
# 'batch_size'参数指定每个批次包含多少样本，'shuffle=True'表示在每个时期结束时重新排列数据以增加随机性。
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

让我们更深入地讨论Normalize操作。

MNIST数据集是灰度图像数据，其像素值范围从0（黑色）到255（白色）。通常在训练神经网络时，将输入特征标准化到均值为0、标准差为1的分布中会有益处。这样可以使得模型更容易地学习并收敛。

`transforms.Normalize(mean = (...), std = (...))`是PyTorch提供的一个函数，用于标准化输入数据。它接受两个参数：均值和标准差。在这段代码中，我们传递了MNIST训练集的均值和标准差（分别为0.1307和0.3081）。

均值和标准差通常是根据训练数据集计算得出的统计量。在上述代码中，提供了MNIST训练集的均值（0.1307）和标准差（0.3081）。

这些值用于标准化训练和测试数据集，以使它们具有相似的分布。在实践中，计算这些值通常是在加载数据之前完成的，因为我们希望确保测试数据集与训练数据集具有相同的统计属性（即，均值和标准差）。

计算这些值时，一般会将所有训练样本的像素值组合成一个向量，并对其进行如下计算：

- 均值：`mean = sum(pixels) / num_pixels`
- 标准差：`std = sqrt(sum((pixel - mean)^2) / num_pixels)`

这样，我们可以确保训练和测试数据集在不同批次中具有相似的分布。使用正确计算的均值和标准差对输入进行标准化是神经网络训练的一种常见技术，它有助于模型更快地收敛并获得较好的性能。

Normalize操作对图像数据进行以下转换：

`normalized_image = (image - mean) / std`

其中，'image'是原始图像的每个像素值，'mean'和'std'分别是我们提供的均值和标准差。这样的转换将数据集的所有特征（在这种情况下是像素）缩放到均值为0、标准差为1的范围内。

这个操作对于训练模型非常重要，因为不同特征的尺度可能会对优化算法产生负面影响。例如，如果一个特征的范围在0到100，而另一个特征的范围在0到1，那么具有更大范围的特征将对训练过程起主导作用，因为优化算法（如梯度下降）会尝试平等地处理所有特征。

通过标准化数据集，我们可以确保每个特征在优化期间都被考虑到相同程度，从而使模型更容易学习并提高其性能。这也有助于加速训练过程。

In [4]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        # self.layer1: 这是一个包含多个层次操作的序列，包括卷积层（Conv2d，输入通道为1，输出通道为6，使用5x5的卷积核，步幅为1）、
        # 批归一化层（BatchNorm2d，对6个通道进行批处理归一化）、激活函数ReLU和最大池化层（MaxPool2d，使用2x2的池化窗口，步幅为2）。

        # 在本层中，我们使用的池化方式是最大池化方式。池化的几种常见方法包括：平均池化、最大池化、K-max池化。
        # 最大池化： 从输入特征图的某个区域子块中选择值最大的像素点作为最大池化结果。
        # 平均池化： 从输入特征图的某个区域子块中选择值平均的像素点作为最大池化结果。
        # K-max池化： 在K个通道上进行最大池化，然后将每个通道上的最大池化结果连接起来作为输出。

        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        # self.layer2: 这是第二个包含多个层次操作的序列，类似于前一层但是输入通道数变成了6（因为上一层有6个输出通道），而输出通道数变成了16。

        
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        # 接下来定义了三个全连接层(self.fc, self.fc1, self.fc2)和两个ReLU激活函数。
        # 全连接层用于将前一层的输出映射到一个特定维度的空间，以便进行分类或回归任务。
        # 这里，第一个全连接层将400个特征（由卷积和池化操作得来）转换为120个特征，
        # 第二个将120个特征转换为84个特征，最后的全连接层将84个特征映射到指定数量的类别上（这就是num_classes参数）
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
    # forward方法定义了前向传播过程，即如何使用输入数据来计算模型的预测结果。
    # 它首先通过第一层和第二层处理输入数据，然后将这些数据展平成一个一维向量（reshape操作），
    # 接着通过全连接层、ReLU激活函数以及最终的预测层计算输出。

In [5]:
# 创建了一个 LeNet5 模型的实例，并将其移动到了设备（可能是 CPU 或 GPU）上。
model = LeNet5(num_classes).to(device)

# 这里定义了损失函数为交叉熵损失（cross-entropy loss）
# 交叉熵刻画了两个概率分布之间的距离，旨在描绘通过概率分布 𝑞 来表达概率分布 𝑝 的困难程度。
# 根据公式不难理解，交叉熵越小，两个概率分布 𝑝 和 𝑞 越接近。
# 很显然，一个良好的神经网络要尽量保证对于每一个输入数据，
# 神经网络所预测类别分布概率与实际类别分布概率之间的差距越小越好，即交叉熵越小越好。
# 于是，可将交叉熵作为损失函数来训练神经网络。
cost = nn.CrossEntropyLoss()

# 这里使用 Adam 优化器来更新模型的权重。model.parameters() 返回模型中所有可学习参数的迭代器，并将它们传递给优化器。
# lr=learning_rate 指定了学习率（learning rate），即每次更新权重时移动的步长大小。
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 这行代码计算训练数据加载器中的样本数量，以便在训练过程中可以知道剩余的步骤数
total_step = len(train_loader)

In [6]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    # 在当前时期内，循环遍历训练数据加载器中的每个批次（即一组图像和相应的标签。
    # enumerate()函数用于跟踪当前的批次索引i。
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #对模型传递当前批次的图像并获得预测输出。
        # 向前传播
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        # 清除优化器中累积的梯度。在执行反向传播之前，这很重要，因为默认情况下PyTorch会累加梯度。
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 使用计算出的梯度更新模型的参数。
        optimizer.step()
        
        # 每 400 批次打印输出一次当前的进度信息。
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.0907
Epoch [1/10], Step [800/938], Loss: 0.0916
Epoch [2/10], Step [400/938], Loss: 0.1063
Epoch [2/10], Step [800/938], Loss: 0.0360
Epoch [3/10], Step [400/938], Loss: 0.0459
Epoch [3/10], Step [800/938], Loss: 0.0269
Epoch [4/10], Step [400/938], Loss: 0.0539
Epoch [4/10], Step [800/938], Loss: 0.1063
Epoch [5/10], Step [400/938], Loss: 0.0020
Epoch [5/10], Step [800/938], Loss: 0.0347
Epoch [6/10], Step [400/938], Loss: 0.0048
Epoch [6/10], Step [800/938], Loss: 0.0082
Epoch [7/10], Step [400/938], Loss: 0.0537
Epoch [7/10], Step [800/938], Loss: 0.0012
Epoch [8/10], Step [400/938], Loss: 0.0043
Epoch [8/10], Step [800/938], Loss: 0.0006
Epoch [9/10], Step [400/938], Loss: 0.0003
Epoch [9/10], Step [800/938], Loss: 0.0004
Epoch [10/10], Step [400/938], Loss: 0.0794
Epoch [10/10], Step [800/938], Loss: 0.0179


In [7]:
# 在测试集上评估模型的性能。

# with torch.no_grad(): - 这一行表示在以下代码块中，不需要计算梯度。
# 这是因为测试阶段只关注模型的性能，而不需要进行反向传播和权重更新。 
with torch.no_grad():

    # 初始化了用于计算模型准确率的变量：
    # correct跟踪预测正确的样本数，
    # 而total是数据集中总样本数。
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images) # 使用测试模型对当前批次的图像进行预测。
        _, predicted = torch.max(outputs.data, 1) # 从模型输出中获取预测类别，即具有最高得分的类别。
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.96 %
